# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [15]:
# Write your code below.

%load_ext dotenv
%dotenv 



The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [16]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [17]:
import os
from glob import glob

# Write your code below.
price_data_dir = os.getenv("PRICE_DATA")


parquet_path = glob(os.path.join(price_data_dir, "**/*.parquet"), recursive= True)

parquet_path



['../../05_src/data/prices/CTAS/CTAS_2002/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2005/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2004/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2003/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2010/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2017/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2021/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2019/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2018/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2020/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2016/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2011/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2008/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2006/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2001/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2000/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2007/part.0.parquet

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [22]:
# Write your code below.

df = dd.read_parquet(parquet_path)

df["Close_lag_1"] = df["Close"].shift(1)
df["Adj Close"] = df["Adj Close"].shift(1)

df["returns"] = (df["Close"] / df["Close_lag_1"]) - 1

df["hi_lo_range"] = df["High"] - df["Low"]

dd_feat = df
dd_feat


,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,returns,hi_lo_range
npartitions=13078,,,,,,,,,,,
,"datetime64[ns, UTC]",float64,float64,float64,float64,float64,float64,int32,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [19]:
# Write your code below.

df = df.compute()


In [20]:
df["returns_maverage_10"] = df["returns"].rolling(10).mean()
df["returns_maverage_10"]

Ticker
A           NaN
A           NaN
A           NaN
A           NaN
A           NaN
         ...   
ZTS    0.005888
ZTS    0.002542
ZTS    0.003573
ZTS    0.003358
ZTS    0.000994
Name: returns_maverage_10, Length: 3173427, dtype: float64

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?  Answer: No
+ Would it have been better to do it in Dask? Why?
Answer: Yes or it depends on the volume of data sets. Dask is for better performance and scalability. Dask delays execution until needed and optimizes computation.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.